In [1]:
import numpy as np
import pandas as pd
import pyomo.environ as pyo
from measure_optimize import MeasurementOptimizer, DataProcess 
import matplotlib.pyplot as plt

## Input Setting

### All measurements and index

- ads.gas_inlet.F (0)
- ads.gas_outlet.F (1)
- ads.gas_outlet.T (2)
- ads.gas_outlet.z (3)
- des.gas_inlet.F (4)
- des.gas_outlet.F (5)
- des.gas_outlet.T (6)
- des.gas_outlet.z (7)
- ads.T19 (8)
- ads.T23 (9)
- ads.T28 (10)
- ads.z19 (11)
- ads.z23 (12)
- ads.z28 (13)

One-time cost ones: 0,1,2, 4,5,6, 8,9,10

Time-cost ones: 3,7,11,12,13

In [2]:
all_names = ['Ads.gas_inlet.F', 'Ads.gas_outlet.F', 'Ads.gas_outlet.T', 
             'Ads.gas_outlet.z("CO2")', 'Des.gas_inlet.F', 'Des.gas_outlet.F', 
             'Des.gas_outlet.T', 'Des.gas_outlet.z("CO2")', 'Ads.T_g.Value(19,10)', 
             'Ads.T_g.Value(23,10)', 'Ads.T_g.Value(28,10)', 'Ads.z("CO2",19,10)', 
             'Ads.z("CO2",23,10)', 'Ads.z("CO2",28,10)']

In [3]:
Nt = 22
static_ind = [0,1,2,4,5,6,8,9,10]
dynamic_ind = [3,7,11,12,13]

In [4]:
dataObject = DataProcess()
dataObject.read_jaco('./RotaryBed/Q'+str(Nt)+'_scale.csv')
Q = dataObject.stack_Q(static_ind, dynamic_ind, Nt)


jacobian shape: (308, 5)


In [5]:
static_name = [all_names[i] for i in static_ind]
dynamic_name = [all_names[i] for i in dynamic_ind]
static_idx_upd = [0,1,2,3,4,5,6,7,8]
dynamic_idx_upd = [9,10,11,12,13]

num_param = 5

measure_names = static_name + dynamic_name
print(measure_names)

['Ads.gas_inlet.F', 'Ads.gas_outlet.F', 'Ads.gas_outlet.T', 'Des.gas_inlet.F', 'Des.gas_outlet.F', 'Des.gas_outlet.T', 'Ads.T_g.Value(19,10)', 'Ads.T_g.Value(23,10)', 'Ads.T_g.Value(28,10)', 'Ads.gas_outlet.z("CO2")', 'Des.gas_outlet.z("CO2")', 'Ads.z("CO2",19,10)', 'Ads.z("CO2",23,10)', 'Ads.z("CO2",28,10)']


In [6]:
calculator = MeasurementOptimizer(Q, static_idx_upd, dynamic_idx_upd, 
                                  num_param, measure_names=measure_names, verbose=True)

fim_expect = calculator.fim_computation()

print(np.shape(calculator.fim_collection))

(14161, 5, 5)


In [7]:
print(calculator.Sigma_inv[(9,0)])
print(calculator.Q_dynamic_flatten[0])

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[-0.539636670963493, -0.0022541710490597, -0.0498629482614828, 1.9351604614661069, 0.5513282116769436], [-0.3340760407153356, 0.1021652968364613, 0.1016944838159672, 2.3403863441379817, 0.221939518269873], [-0.3494661138571784, 0.1120662029219454, 0.2550896605274281, 2.0027607969742065, 0.1809770476947275], [-0.4678795656821666, 0.0789891670526334, 0.3012354877648384, 1.6852339585553322, 0.3702798007923669], [-0.3487072812830348, 0.1544493767900593, 0.3679533495956177, 2.016853552909431, 0.1889221494208309], [-0.2149867536290262, 0.1729367514893193, 0.4172631902483755, 2.2827109973126425, -0.0128215863177127], [-0.0597343516865485, 0.1787397054563655, 0.4592339471516512, 2.0431098799382568, -0.2109020116805193], [-0.2295705671608305, 0.0878631069081999, 0.393229772557104, 0.9576582753596296, 0.0872533926070839], [-0.1234288608689441, 0.0230056295989378, 

In [8]:
print(calculator.dynamic_to_flatten[1])

[22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43]


### Set up grey-box 

In [9]:
from greybox_generalize import LogDetModel

In [10]:
def build_model_external(m):
    ex_model = LogDetModel(num_para=5)
    m.egb = ExternalGreyBoxBlock()
    m.egb.set_external_model(ex_model)

### Reform the inputs

In [13]:
num_static = len(static_ind)
num_dynamic  = len(dynamic_ind)
num_total = num_static + num_dynamic*Nt

In [15]:
cost = [1000, #ads.gas_inlet.F (0)
        1000, #ads.gas_outlet.F (1)
         200, #ads.gas_outlet.T (2)
        1000, #des.gas_inlet.F (4)
        1000, #des.gas_outlet.F (5)
         200, #des.gas_outlet.T (6)
         600, #ads.T19 (8)
         600, #ads.T23 (9)
         600 #ads.T28 (10)
       ]

for i in range(Nt*num_dynamic):
    cost.append(10)


In [16]:
#print(np.shape(ans_y))

init_y = np.random.rand(num_total)

update_y = [] 

for i in init_y:
    if i > 0.5: 
        update_y.append(1)
    else:
        update_y.append(0)

init_cov_y = [[0]*num_total for i in range(num_total)]

for j in range(num_total):
    for k in range(num_total):
        if j==k:
            init_cov_y[j][k] = update_y[k]
            
        else:
            init_cov_y[j][k] = min(update_y[k], update_y[j])

fim_prior = np.identity((5))
print(fim_prior)

[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


In [17]:
mip_option = False
objective = "D"
budget_opt = 5000
sparse_opt = False
fix_opt = False

In [18]:
mod = calculator.continuous_optimization(cost, mixed_integer=mip_option, 
                  obj=objective, fix=fix_opt, sparse=sparse_opt, budget=budget_opt,
                                        initial_fim = fim_prior)

In [19]:

if not mip_option and objective=="A":
    solver = pyo.SolverFactory('ipopt')
    solver.options['linear_solver'] = "ma57"
    solver.solve(mod, tee=True)

elif mip_option and objective=="A":
    solver = pyo.SolverFactory('mindtpy')
    results = solver.solve(mod, tee=True, integer_tolerance=0.5)

elif objective=="D":  
    solver = pyo.SolverFactory('cyipopt')
    solver.config.options['hessian_approximation'] = 'limited-memory' 
    additional_options={'max_iter':3000}
    for k,v in additional_options.items():
        solver.config.options[k] = v
    results = solver.solve(mod, tee=True)


 Consider M =
 [[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]
   logdet =  0.0 


 Consider M =
 [[ 1.08580694  0.00608449 -0.09632441 -0.05093109 -0.12128496]
 [ 0.00608449  0.95556401  0.0044767  -0.0043866  -0.00548779]
 [-0.09632441  0.0044767   0.96992282  0.03952859  0.1280515 ]
 [-0.05093109 -0.0043866   0.03952859  1.01231402 -0.02707322]
 [-0.12128496 -0.00548779  0.1280515  -0.02707322  1.01945027]]
   logdet =  -0.0038353289796222746 


 Consider M =
 [[ 1.98417920e+00  3.78911470e-02  5.25936821e-01 -1.02106721e-01
  -1.72979489e+00]
 [ 3.78911470e-02  9.24304327e-01  4.87975245e-03  3.45422359e-01
  -5.70574199e-02]
 [ 5.25936821e-01  4.87975245e-03  1.24321945e+00  3.31475612e-02
  -4.64114725e-01]
 [-1.02106721e-01  3.45422359e-01  3.31475612e-02  9.37166181e+00
  -1.18942406e+00]
 [-1.72979489e+00 -5.70574199e-02 -4.64114725e-01 -1.18942406e+00
   3.23986683e+00]]
   logdet =  3.3697475823545755 


 Consider M =
 [[ 1.9602482


 Consider M =
 [[ 1.51980456e+02  3.12945386e-01 -8.99240128e+01  2.60041871e+02
  -3.50132943e+01]
 [ 3.12945386e-01  1.33729464e+00 -3.42398080e-01  1.39746069e+01
  -1.02070802e+00]
 [-8.99240128e+01 -3.42398080e-01  9.27493531e+01 -2.46277823e+02
  -6.82860579e+00]
 [ 2.60041871e+02  1.39746069e+01 -2.46277823e+02  1.06367970e+03
  -3.49714248e+01]
 [-3.50132943e+01 -1.02070802e+00 -6.82860579e+00 -3.49714248e+01
   4.70836941e+01]]
   logdet =  17.37428344099181 


 Consider M =
 [[ 1.64739214e+02  3.75415970e-01 -9.46914185e+01  2.78613889e+02
  -4.09179552e+01]
 [ 3.75415970e-01  1.38174835e+00 -3.29267277e-01  1.43606199e+01
  -1.12965656e+00]
 [-9.46914185e+01 -3.29267277e-01  9.99989517e+01 -2.62027009e+02
  -9.37824477e+00]
 [ 2.78613889e+02  1.43606199e+01 -2.62027009e+02  1.12504603e+03
  -3.83900194e+01]
 [-4.09179552e+01 -1.12965656e+00 -9.37824477e+00 -3.83900194e+01
   5.53915550e+01]]
   logdet =  17.772987718986716 


 Consider M =
 [[ 2.89294243e+02  3.99539601e-01


 Consider M =
 [[ 4.38080500e+03  8.82672755e-01 -3.36646400e+03  8.80319749e+03
  -2.77974266e+02]
 [ 8.82672755e-01  2.25652671e+00 -3.04612810e-01  2.39406535e+01
  -1.41265761e+00]
 [-3.36646400e+03 -3.04612810e-01  2.66974125e+03 -6.90979409e+03
   1.53034607e+02]
 [ 8.80319749e+03  2.39406535e+01 -6.90979409e+03  1.95552591e+04
  -4.72944044e+02]
 [-2.77974266e+02 -1.41265761e+00  1.53034607e+02 -4.72944044e+02
   1.03362699e+02]]
   logdet =  24.552735478748495 


 Consider M =
 [[ 4.81379707e+03  9.21045846e-01 -3.70196586e+03  9.67831877e+03
  -3.02627137e+02]
 [ 9.21045846e-01  2.32445495e+00 -2.79483735e-01  2.45819973e+01
  -1.38858406e+00]
 [-3.70196586e+03 -2.79483735e-01  2.93366807e+03 -7.59098454e+03
   1.68999597e+02]
 [ 9.67831877e+03  2.45819973e+01 -7.59098454e+03  2.14326452e+04
  -5.16560691e+02]
 [-3.02627137e+02 -1.38858406e+00  1.68999597e+02 -5.16560691e+02
   1.08678312e+02]]
   logdet =  24.83333540358786 


 Consider M =
 [[ 5.06164923e+03  9.37817538e-01

In [ ]:
### FIM
fim_result = np.zeros((5,5))
for i in range(5):
    for j in range(5):
        fim_result[i,j] = pyo.value(mod.TotalFIM[i,j])
        
print(fim_result)  
print('trace:', np.trace(fim_result))
print('det:', np.linalg.det(fim_result))

print("Pyomo OF:", pyo.value(mod.Obj))
print("Log_det:", np.log(np.linalg.det(fim_result)))

In [ ]:
# problem size calculator
if True:
    nr = num_total
    npa = 5
    num_var = (nr*nr+nr)/2 + npa*npa + 1
    num_equal = npa*npa+1
    num_ineq = 1.5*(nr**2-nr)+1

    print(num_var, num_equal, num_ineq)

In [ ]:
import numpy as np

ans_y = np.zeros((num_total,num_total))

pyomo_y = np.zeros((num_total, num_total))

for i in range(num_total):
    for j in range(i, num_total):
        cov = pyo.value(mod.cov_y[i,j])
        ans_y[i,j] = cov 
        ans_y[j,i] = pyo.value(mod.cov_y[j,i])

for i in range(num_total):
    for j in range(num_total):
        pyomo_y[i,j] = pyo.value(mod.cov_y[i,j])
        
print("Cost for this stragy:", pyo.value(mod.cost))
print("Cost if no budget:", sum(cost))

In [ ]:
for i in range(len(ans_y)):
    for j in range(len(ans_y[0])):
        if ans_y[i][j] < 0.05:
            ans_y[i][j] = int(0)
        elif ans_y[i][j] > 0.95:
            ans_y[i][j] = int(1)
        else: 
            ans_y[i][j] = round(ans_y[i][j], 2)
print(ans_y)

In [ ]:
import pickle

store = False

if store:
    file = open('Fe23_D_110_3000', 'wb')

    pickle.dump(ans_y, file)

    file.close()

In [ ]:
for c in range(num_static):
    print(static_name[c], ": ", ans_y[c,c])

sol_y = np.asarray([ans_y[i,i] for i in range(num_static, num_total)])

sol_y = np.reshape(sol_y, (num_dynamic, dynamic_Nt))
np.around(sol_y)

for r in range(len(sol_y)):
    print(dynamic_name[r], ": ", sol_y[r])